In [97]:
import pandas as pd

In [98]:
embds_file = "../data/embds/glove.840B.300d.txt"
# source_name = "../data/snli_1.0/cl_snli_train_source_file"
source_name = "../data/multinli_1.0/cl_multinli_train_source_file"
labels = "../data/multinli_1.0/cl_multinli_train_lbl_file"


output = open("mnli_train_glovefixed.txt", 'w')
output_labels = open("mnli_train_glovefixed_labels.txt", 'w')

In [99]:
def add_word(w, d):
    if w in d:
        d[w] += 1
    else:
        d[w] = 1

In [100]:
words_missing = []
if "multinli" in source_name:
    source = open(source_name, 'rb')
else:
    source = open(source_name, 'r')
vocab = set()
words_affected = {}
labels = open(labels, 'r')

In [101]:
missing_words = set()
vectored_words = set()
samples_affected = 0
affected_num = 0
with open(embds_file, encoding="utf8") as f:
    for line in f:
      word, vec = line.split(' ', 1)
      vectored_words.add(word)

In [102]:
num_samples = 0
dropped_words = 0

new_source = []
new_labels = []

try:
    while True:
        label = next(labels).replace("\n", "")
        if "multinli" in source_name:
            try:
                sample = next(source).decode(encoding='UTF-8')
                line = sample.replace("\n", "").replace(" .", "").split("|||")
            except UnicodeDecodeError:
                pass
        else:
            sample = next(source)
            line = sample.replace("\n", "").replace(" .", "").split("|||")
        premise, hypothesis = line[0].split(" "), line[1].split(" ")
        affected = False
        for word in hypothesis:
            if word not in vectored_words:
                add_word(word, words_affected)
                affected_num += 1
                affected = True
            vocab.add(word)
        for word in premise:
            if word not in vectored_words:
                add_word(word, words_affected)
                affected_num += 1
                affected = True
            vocab.add(word)
        if affected:
            samples_affected += 1
        else:
            new_source.append(sample)
            new_labels.append(label + "\n")
        num_samples += 1
        if num_samples % 20000 == 0:
            print(num_samples, "completed")
except StopIteration:
    pass

print(len(new_source), len(new_labels), num_samples - len(new_labels), samples_affected)
new_source = "".join(new_source)
new_labels = "".join(new_labels)

20000 completed
40000 completed
60000 completed
80000 completed
100000 completed
120000 completed
140000 completed
160000 completed
180000 completed
200000 completed
220000 completed
240000 completed
260000 completed
280000 completed
300000 completed
320000 completed
340000 completed
360000 completed
380000 completed
343632 343632 48976 48976


In [103]:
output.write(new_source)
output_labels.write(new_labels)

3779433

In [104]:
missing_words = vocab - vectored_words
len(missing_words)

15481

In [105]:
len(missing_words)/len(vocab)

0.15109605887290403

In [106]:
samples_affected

48976

In [107]:
affected_num

71827

In [108]:
num_samples

392608

In [109]:
affected_num/num_samples

0.1829483861765425

In [110]:
words = pd.DataFrame.from_dict(words_affected, orient='index').reset_index().rename(columns={0: "num", "index": "word"}).sort_values(by="num", ascending=False)
words.head(20)

,word,num
9,Ca'daan,2581
31,um-hum,2106
116,San'doro,926
50,Vandemeyer,712
131,'I,612
137,Karf,349
398,Hersheimmer,321
454,Bauerstein,284
49,'You,245
395,A'deem,238


In [111]:
numericals = []

for word in missing_words:
    for num in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
        if num in word:
            numericals.append(word)
            continue

In [112]:
len(numericals)

4274